In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

# Statistical Models
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# Model Selection
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import itertools

ModuleNotFoundError: No module named 'statsmodels'

In [2]:



# ============================================================
# DATA FETCH
# ============================================================
def fetch_data(ticker: str, start_date='2012-01-01') -> pd.DataFrame:
    """Fetch historical stock data"""
    end_date = datetime.now().strftime('%Y-%m-%d')
    df = yf.download(ticker, start=start_date, end=end_date)
    
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [c[0] for c in df.columns]
    
    # Use only Close price for time series
    df = df[['Close']].copy()
    df.columns = ['Close']
    
    return df


# ============================================================
# TIME SERIES ANALYSIS
# ============================================================
def check_stationarity(series, name="Series"):
    """
    Perform Augmented Dickey-Fuller test to check stationarity
    """
    result = adfuller(series.dropna())
    
    print(f"\n{'='*60}")
    print(f"STATIONARITY TEST: {name}")
    print(f"{'='*60}")
    print(f"ADF Statistic: {result[0]:.6f}")
    print(f"P-value: {result[1]:.6f}")
    print(f"Critical Values:")
    for key, value in result[4].items():
        print(f"  {key}: {value:.3f}")
    
    if result[1] <= 0.05:
        print(f"\n✅ {name} is STATIONARY (p-value ≤ 0.05)")
        return True
    else:
        print(f"\n❌ {name} is NON-STATIONARY (p-value > 0.05)")
        print(f"   → Need differencing to make it stationary")
        return False


def make_stationary(df, max_diff=2):
    """
    Apply differencing to make series stationary
    """
    df_work = df.copy()
    
    for d in range(max_diff + 1):
        if d == 0:
            series = df_work['Close']
            name = "Original Series"
        else:
            series = df_work['Close'].diff(d).dropna()
            name = f"After {d} Differencing"
        
        is_stationary = check_stationarity(series, name)
        
        if is_stationary:
            return d, series
    
    print(f"\n⚠️ Could not achieve stationarity with {max_diff} differencing")
    return max_diff, series


def seasonal_decomposition(df):
    """
    Decompose time series into trend, seasonal, and residual components
    """
    # Use multiplicative model for stock prices
    decomposition = seasonal_decompose(
        df['Close'], 
        model='multiplicative', 
        period=252  # ~1 trading year
    )
    
    fig = make_subplots(
        rows=4, cols=1,
        subplot_titles=['Original', 'Trend', 'Seasonal', 'Residual'],
        vertical_spacing=0.08
    )
    
    fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name='Original'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=decomposition.trend, name='Trend'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=decomposition.seasonal, name='Seasonal'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=decomposition.resid, name='Residual'), row=4, col=1)
    
    fig.update_layout(
        height=900,
        title_text="Seasonal Decomposition of Stock Price",
        showlegend=False,
        template='plotly_dark'
    )
    
    return fig


def plot_acf_pacf(series, lags=40):
    """
    Plot ACF and PACF to determine ARIMA parameters
    """
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    plot_acf(series.dropna(), lags=lags, ax=axes[0])
    axes[0].set_title('Autocorrelation Function (ACF)')
    
    plot_pacf(series.dropna(), lags=lags, ax=axes[1])
    axes[1].set_title('Partial Autocorrelation Function (PACF)')
    
    plt.tight_layout()
    return fig


# ============================================================
# AUTO ARIMA - AUTOMATIC PARAMETER SELECTION
# ============================================================
def find_best_arima(series, seasonal=False, m=1):
    """
    Use auto_arima to find optimal parameters
    """
    print(f"\n{'='*60}")
    print("AUTO ARIMA: Finding Optimal Parameters...")
    print(f"{'='*60}")
    
    model = auto_arima(
        series.dropna(),
        start_p=0, start_q=0,
        max_p=5, max_q=5,
        d=None,  # Let auto_arima determine d
        seasonal=seasonal,
        start_P=0, start_Q=0,
        max_P=2, max_Q=2,
        m=m,  # Seasonal period
        trace=True,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True,
        random_state=42,
        n_fits=50
    )
    
    print(f"\n✅ BEST MODEL: {model.order}")
    if seasonal:
        print(f"   Seasonal Order: {model.seasonal_order}")
    print(f"   AIC: {model.aic():.2f}")
    print(f"   BIC: {model.bic():.2f}")
    
    return model


# ============================================================
# ARIMA MODEL
# ============================================================
def build_arima_model(df, order=None, test_size=0.15):
    """
    Build and train ARIMA model
    """
    # Split data
    split = int(len(df) * (1 - test_size))
    train = df.iloc[:split]
    test = df.iloc[split:]
    
    print(f"\n{'='*60}")
    print("ARIMA MODEL TRAINING")
    print(f"{'='*60}")
    print(f"Training samples: {len(train)}")
    print(f"Testing samples: {len(test)}")
    
    # Auto-find parameters if not provided
    if order is None:
        auto_model = find_best_arima(train['Close'], seasonal=False)
        order = auto_model.order
    
    print(f"\nTraining ARIMA{order}...")
    
    # Build ARIMA model
    model = ARIMA(train['Close'], order=order)
    fitted_model = model.fit()
    
    print(f"\n📊 Model Summary:")
    print(f"   AIC: {fitted_model.aic:.2f}")
    print(f"   BIC: {fitted_model.bic:.2f}")
    
    # Make predictions on test set
    predictions = fitted_model.forecast(steps=len(test))
    
    # Calculate metrics
    mse = mean_squared_error(test['Close'], predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test['Close'], predictions)
    mape = np.mean(np.abs((test['Close'] - predictions) / test['Close'])) * 100
    
    # Direction accuracy
    actual_direction = np.sign(test['Close'].diff().dropna())
    pred_direction = np.sign(pd.Series(predictions).diff().dropna())
    direction_acc = np.mean(actual_direction.values == pred_direction.values) * 100
    
    metrics = {
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'direction_accuracy': direction_acc
    }
    
    return fitted_model, train, test, predictions, metrics


# ============================================================
# SARIMA MODEL
# ============================================================
def build_sarima_model(df, order=None, seasonal_order=None, test_size=0.15, m=5):
    """
    Build and train SARIMA model with seasonality
    m = seasonal period (5 for weekly, 21 for monthly, 252 for yearly)
    """
    # Split data
    split = int(len(df) * (1 - test_size))
    train = df.iloc[:split]
    test = df.iloc[split:]
    
    print(f"\n{'='*60}")
    print("SARIMA MODEL TRAINING")
    print(f"{'='*60}")
    print(f"Training samples: {len(train)}")
    print(f"Testing samples: {len(test)}")
    print(f"Seasonal period (m): {m}")
    
    # Auto-find parameters if not provided
    if order is None or seasonal_order is None:
        auto_model = find_best_arima(train['Close'], seasonal=True, m=m)
        order = auto_model.order
        seasonal_order = auto_model.seasonal_order
    
    print(f"\nTraining SARIMA{order}x{seasonal_order}...")
    
    # Build SARIMA model
    model = SARIMAX(
        train['Close'],
        order=order,
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    fitted_model = model.fit(disp=False)
    
    print(f"\n📊 Model Summary:")
    print(f"   AIC: {fitted_model.aic:.2f}")
    print(f"   BIC: {fitted_model.bic:.2f}")
    
    # Make predictions on test set
    predictions = fitted_model.forecast(steps=len(test))
    
    # Calculate metrics
    mse = mean_squared_error(test['Close'], predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test['Close'], predictions)
    mape = np.mean(np.abs((test['Close'] - predictions) / test['Close'])) * 100
    
    # Direction accuracy
    actual_direction = np.sign(test['Close'].diff().dropna())
    pred_direction = np.sign(pd.Series(predictions).diff().dropna())
    direction_acc = np.mean(actual_direction.values == pred_direction.values) * 100
    
    metrics = {
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'direction_accuracy': direction_acc
    }
    
    return fitted_model, train, test, predictions, metrics


# ============================================================
# GRID SEARCH FOR BEST PARAMETERS
# ============================================================
def grid_search_arima(df, p_range=(0, 3), d_range=(0, 2), q_range=(0, 3)):
    """
    Manual grid search for ARIMA parameters
    """
    print(f"\n{'='*60}")
    print("GRID SEARCH: Finding Best ARIMA Parameters")
    print(f"{'='*60}")
    
    split = int(len(df) * 0.85)
    train = df.iloc[:split]
    
    best_aic = np.inf
    best_order = None
    results = []
    
    # Generate all combinations
    pdq = list(itertools.product(
        range(p_range[0], p_range[1] + 1),
        range(d_range[0], d_range[1] + 1),
        range(q_range[0], q_range[1] + 1)
    ))
    
    total = len(pdq)
    print(f"Testing {total} parameter combinations...\n")
    
    for i, (p, d, q) in enumerate(pdq, 1):
        try:
            model = ARIMA(train['Close'], order=(p, d, q))
            fitted = model.fit()
            
            results.append({
                'order': (p, d, q),
                'aic': fitted.aic,
                'bic': fitted.bic
            })
            
            if fitted.aic < best_aic:
                best_aic = fitted.aic
                best_order = (p, d, q)
            
            print(f"[{i}/{total}] ARIMA{(p,d,q)} → AIC: {fitted.aic:.2f}")
            
        except:
            continue
    
    print(f"\n✅ BEST MODEL: ARIMA{best_order}")
    print(f"   AIC: {best_aic:.2f}")
    
    return best_order, pd.DataFrame(results).sort_values('aic')


# ============================================================
# VISUALIZATION
# ============================================================
def plot_predictions(train, test, predictions, model_name="ARIMA"):
    """
    Plot actual vs predicted values
    """
    fig = go.Figure()
    
    # Training data
    fig.add_trace(go.Scatter(
        x=train.index,
        y=train['Close'],
        name='Training Data',
        line=dict(color='#3b82f6', width=2)
    ))
    
    # Actual test data
    fig.add_trace(go.Scatter(
        x=test.index,
        y=test['Close'],
        name='Actual',
        line=dict(color='#10b981', width=2)
    ))
    
    # Predictions
    fig.add_trace(go.Scatter(
        x=test.index,
        y=predictions,
        name='Predicted',
        line=dict(color='#ef4444', width=2, dash='dash')
    ))
    
    fig.update_layout(
        title=f"{model_name} Model: Actual vs Predicted",
        xaxis_title="Date",
        yaxis_title="Price (₹)",
        template="plotly_dark",
        hovermode='x unified',
        height=600
    )
    
    return fig


def plot_forecast(df, fitted_model, forecast_days=60, model_name="ARIMA"):
    """
    Plot future forecast
    """
    # Generate forecast
    forecast = fitted_model.forecast(steps=forecast_days)
    forecast_index = pd.date_range(
        start=df.index[-1] + pd.Timedelta(days=1),
        periods=forecast_days
    )
    
    # Get confidence intervals (if available)
    try:
        forecast_df = fitted_model.get_forecast(steps=forecast_days)
        conf_int = forecast_df.conf_int()
        has_conf = True
    except:
        has_conf = False
    
    fig = go.Figure()
    
    # Historical data (last 180 days)
    fig.add_trace(go.Scatter(
        x=df.index[-180:],
        y=df['Close'].iloc[-180:],
        name='Historical',
        line=dict(color='#3b82f6', width=2)
    ))
    
    # Forecast
    fig.add_trace(go.Scatter(
        x=forecast_index,
        y=forecast,
        name=f'{forecast_days}-Day Forecast',
        line=dict(color='#f59e0b', width=3, dash='dash')
    ))
    
    # Confidence intervals
    if has_conf:
        fig.add_trace(go.Scatter(
            x=forecast_index,
            y=conf_int.iloc[:, 0],
            fill=None,
            mode='lines',
            line=dict(color='rgba(245, 158, 11, 0.2)'),
            showlegend=False
        ))
        
        fig.add_trace(go.Scatter(
            x=forecast_index,
            y=conf_int.iloc[:, 1],
            fill='tonexty',
            mode='lines',
            line=dict(color='rgba(245, 158, 11, 0.2)'),
            name='95% Confidence Interval'
        ))
    
    fig.update_layout(
        title=f"{model_name} Model: {forecast_days}-Day Forecast",
        xaxis_title="Date",
        yaxis_title="Price (₹)",
        template="plotly_dark",
        hovermode='x unified',
        height=600
    )
    
    return fig, forecast


def plot_residuals(fitted_model, model_name="ARIMA"):
    """
    Plot residual diagnostics
    """
    residuals = fitted_model.resid
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Residuals Over Time',
            'Residual Histogram',
            'Q-Q Plot',
            'ACF of Residuals'
        ]
    )
    
    # Time series of residuals
    fig.add_trace(
        go.Scatter(y=residuals, mode='lines', name='Residuals'),
        row=1, col=1
    )
    
    # Histogram
    fig.add_trace(
        go.Histogram(x=residuals, name='Distribution', nbinsx=50),
        row=1, col=2
    )
    
    # Q-Q Plot
    from scipy import stats
    qq = stats.probplot(residuals, dist="norm")
    fig.add_trace(
        go.Scatter(x=qq[0][0], y=qq[0][1], mode='markers', name='Q-Q'),
        row=2, col=1
    )
    # Add reference line
    fig.add_trace(
        go.Scatter(x=qq[0][0], y=qq[0][0], mode='lines', name='Normal', line=dict(dash='dash')),
        row=2, col=1
    )
    
    # ACF of residuals
    acf_vals = acf(residuals, nlags=40)
    fig.add_trace(
        go.Bar(y=acf_vals, name='ACF'),
        row=2, col=2
    )
    
    fig.update_layout(
        title_text=f"{model_name} Residual Diagnostics",
        showlegend=False,
        height=800,
        template='plotly_dark'
    )
    
    return fig


# ============================================================
# COMPARISON FUNCTION
# ============================================================
def compare_models(df, test_size=0.15):
    """
    Compare ARIMA and SARIMA models
    """
    print(f"\n{'='*60}")
    print("MODEL COMPARISON: ARIMA vs SARIMA")
    print(f"{'='*60}")
    
    # Train ARIMA
    arima_model, train, test, arima_pred, arima_metrics = build_arima_model(df, test_size=test_size)
    
    # Train SARIMA (weekly seasonality)
    sarima_model, _, _, sarima_pred, sarima_metrics = build_sarima_model(df, test_size=test_size, m=5)
    
    # Print comparison
    print(f"\n{'='*60}")
    print("PERFORMANCE COMPARISON")
    print(f"{'='*60}")
    print(f"\n{'Metric':<20} {'ARIMA':>15} {'SARIMA':>15}")
    print(f"{'-'*50}")
    print(f"{'RMSE':<20} ₹{arima_metrics['rmse']:>14.2f} ₹{sarima_metrics['rmse']:>14.2f}")
    print(f"{'MAE':<20} ₹{arima_metrics['mae']:>14.2f} ₹{sarima_metrics['mae']:>14.2f}")
    print(f"{'MAPE (%)':<20} {arima_metrics['mape']:>14.2f}% {sarima_metrics['mape']:>14.2f}%")
    print(f"{'Direction Acc (%)':<20} {arima_metrics['direction_accuracy']:>14.2f}% {sarima_metrics['direction_accuracy']:>14.2f}%")
    
    # Determine winner
    if arima_metrics['rmse'] < sarima_metrics['rmse']:
        print(f"\n🏆 Winner: ARIMA (Lower RMSE)")
        best_model = arima_model
        best_name = "ARIMA"
    else:
        print(f"\n🏆 Winner: SARIMA (Lower RMSE)")
        best_model = sarima_model
        best_name = "SARIMA"
    
    return {
        'arima_model': arima_model,
        'sarima_model': sarima_model,
        'best_model': best_model,
        'best_name': best_name,
        'arima_metrics': arima_metrics,
        'sarima_metrics': sarima_metrics,
        'test': test,
        'arima_predictions': arima_pred,
        'sarima_predictions': sarima_pred
    }


# ============================================================
# PRINT EVALUATION METRICS
# ============================================================
def print_metrics(metrics, model_name="Model"):
    """
    Print evaluation metrics in a formatted way
    """
    print(f"\n{'='*60}")
    print(f"{model_name} EVALUATION METRICS")
    print(f"{'='*60}")
    print(f"📊 RMSE: ₹{metrics['rmse']:.2f}")
    print(f"📉 MAE: ₹{metrics['mae']:.2f}")
    print(f"📈 MAPE: {metrics['mape']:.2f}%")
    print(f"🎯 Direction Accuracy: {metrics['direction_accuracy']:.2f}%")
    print(f"{'='*60}")

In [5]:
def example_1_basic_arima():
    """Simple ARIMA forecasting"""
    
    # 1. Fetch data
    ticker = "RELIANCE.NS"
    df = fetch_data(ticker, start_date='2020-01-01')
    print(f"✅ Fetched {len(df)} days of data for {ticker}")
    
    # 2. Check stationarity
    d_order, stationary_series = make_stationary(df)
    
    # 3. Build ARIMA model (auto-finds best parameters)
    model, train, test, predictions, metrics = build_arima_model(df)
    
    # 4. Print metrics
    print_metrics(metrics, "ARIMA")
    
    # 5. Visualize results
    fig = plot_predictions(train, test, predictions, "ARIMA")
    fig.show()
    
    # 6. Future forecast
    fig_forecast, forecast = plot_forecast(df, model, forecast_days=30, model_name="ARIMA")
    fig_forecast.show()
    
    print(f"\n📅 30-Day Forecast:")
    print(f"   Current Price: ₹{df['Close'].iloc[-1]:.2f}")
    print(f"   Predicted (30 days): ₹{forecast.iloc[-1]:.2f}")
    print(f"   Expected Change: {((forecast.iloc[-1] / df['Close'].iloc[-1] - 1) * 100):.2f}%")


# ============================================================
# EXAMPLE 2: SARIMA MODEL WITH SEASONALITY
# ============================================================
def example_2_sarima():
    """SARIMA model with weekly seasonality"""
    
    ticker = "TCS.NS"
    df = fetch_data(ticker, start_date='2020-01-01')
    
    # Build SARIMA with weekly seasonality (m=5 trading days)
    model, train, test, predictions, metrics = build_sarima_model(
        df, 
        m=5,  # Weekly seasonality
        test_size=0.15
    )
    
    print_metrics(metrics, "SARIMA")
    
    # Visualize
    fig = plot_predictions(train, test, predictions, "SARIMA")
    fig.show()
    
    # Residual analysis
    fig_resid = plot_residuals(model, "SARIMA")
    fig_resid.show()


# ============================================================
# EXAMPLE 3: COMPARE ARIMA vs SARIMA
# ============================================================
def example_3_compare_models():
    """Compare ARIMA and SARIMA performance"""
    
    ticker = "INFY.NS"
    df = fetch_data(ticker, start_date='2020-01-01')
    
    # Compare both models
    results = compare_models(df, test_size=0.15)
    
    # Visualize best model
    if results['best_name'] == 'ARIMA':
        pred = results['arima_predictions']
    else:
        pred = results['sarima_predictions']
    
    # Plot comparison
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=results['test'].index,
        y=results['test']['Close'],
        name='Actual',
        line=dict(color='#10b981', width=3)
    ))
    
    fig.add_trace(go.Scatter(
        x=results['test'].index,
        y=results['arima_predictions'],
        name='ARIMA',
        line=dict(color='#3b82f6', width=2, dash='dash')
    ))
    
    fig.add_trace(go.Scatter(
        x=results['test'].index,
        y=results['sarima_predictions'],
        name='SARIMA',
        line=dict(color='#f59e0b', width=2, dash='dot')
    ))
    
    fig.update_layout(
        title="ARIMA vs SARIMA Comparison",
        xaxis_title="Date",
        yaxis_title="Price (₹)",
        template="plotly_dark",
        hovermode='x unified'
    )
    fig.show()
    
    # Future forecast with best model
    fig_forecast, forecast = plot_forecast(
        df, 
        results['best_model'], 
        forecast_days=60, 
        model_name=results['best_name']
    )
    fig_forecast.show()


# ============================================================
# EXAMPLE 4: MANUAL PARAMETER TUNING
# ============================================================
def example_4_manual_parameters():
    """Use specific ARIMA parameters"""
    
    ticker = "HDFCBANK.NS"
    df = fetch_data(ticker, start_date='2020-01-01')
    
    # Option 1: Use specific parameters
    model, train, test, predictions, metrics = build_arima_model(
        df,
        order=(2, 1, 2)  # p=2, d=1, q=2
    )
    
    print_metrics(metrics, "ARIMA(2,1,2)")
    
    # Option 2: Grid search for best parameters
    best_order, results_df = grid_search_arima(
        df,
        p_range=(0, 3),
        d_range=(0, 2),
        q_range=(0, 3)
    )
    
    print("\nTop 5 Models by AIC:")
    print(results_df.head())
    
    # Train with best parameters
    model, train, test, predictions, metrics = build_arima_model(
        df,
        order=best_order
    )


# ============================================================
# EXAMPLE 5: SEASONAL DECOMPOSITION
# ============================================================
def example_5_seasonal_analysis():
    """Analyze seasonal patterns before modeling"""
    
    ticker = "TATAMOTORS.NS"
    df = fetch_data(ticker, start_date='2018-01-01')  # Need more data for seasonality
    
    # Decompose time series
    fig_decomp = seasonal_decomposition(df)
    fig_decomp.show()
    
    # Check ACF and PACF
    fig_acf = plot_acf_pacf(df['Close'])
    plt.show()
    
    # Based on seasonality, choose appropriate model
    # If strong seasonal pattern → use SARIMA
    # If weak/no seasonality → use ARIMA


# ============================================================
# EXAMPLE 6: MULTI-HORIZON FORECASTING
# ============================================================
def example_6_multi_horizon():
    """Forecast multiple time horizons"""
    
    ticker = "WIPRO.NS"
    df = fetch_data(ticker, start_date='2020-01-01')
    
    # Train model
    model, _, _, _, metrics = build_arima_model(df)
    
    print(f"\n{'='*60}")
    print("MULTI-HORIZON FORECASTING")
    print(f"{'='*60}")
    
    current_price = df['Close'].iloc[-1]
    
    horizons = [7, 15, 30, 60, 90]
    for days in horizons:
        forecast = model.forecast(steps=days)
        predicted_price = forecast.iloc[-1]
        change_pct = ((predicted_price / current_price - 1) * 100)
        
        print(f"\n📅 {days:3d}-Day Forecast:")
        print(f"   Price: ₹{predicted_price:.2f}")
        print(f"   Change: {change_pct:+.2f}%")


# ============================================================
# EXAMPLE 7: WALK-FORWARD VALIDATION
# ============================================================
def example_7_walk_forward():
    """More robust validation using walk-forward approach"""
    
    ticker = "SBIN.NS"
    df = fetch_data(ticker, start_date='2020-01-01')
    
    # Parameters
    train_size = int(len(df) * 0.8)
    test_size = len(df) - train_size
    
    print(f"\n{'='*60}")
    print("WALK-FORWARD VALIDATION")
    print(f"{'='*60}")
    
    predictions = []
    actuals = []
    
    for i in range(test_size):
        # Rolling window
        train = df.iloc[:train_size + i]
        
        # Train model
        model = ARIMA(train['Close'], order=(1, 1, 1))
        fitted = model.fit()
        
        # One-step ahead forecast
        forecast = fitted.forecast(steps=1)
        predictions.append(forecast.iloc[0])
        actuals.append(df['Close'].iloc[train_size + i])
        
        if (i + 1) % 10 == 0:
            print(f"Completed {i+1}/{test_size} forecasts...")
    
    # Calculate metrics
    predictions = np.array(predictions)
    actuals = np.array(actuals)
    
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    mae = mean_absolute_error(actuals, predictions)
    
    print(f"\n📊 Walk-Forward Results:")
    print(f"   RMSE: ₹{rmse:.2f}")
    print(f"   MAE: ₹{mae:.2f}")


# ============================================================
# EXAMPLE 8: SAVE AND LOAD MODEL
# ============================================================
def example_8_save_load():
    """Save trained model for later use"""
    
    import pickle
    
    ticker = "ITC.NS"
    df = fetch_data(ticker, start_date='2020-01-01')
    
    # Train model
    model, _, _, _, _ = build_arima_model(df)
    
    # Save model
    with open(f'{ticker}_arima_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    print(f"\n✅ Model saved: {ticker}_arima_model.pkl")
    
    # Load model
    with open(f'{ticker}_arima_model.pkl', 'rb') as f:
        loaded_model = pickle.load(f)
    print(f"✅ Model loaded successfully")
    
    # Make forecast with loaded model
    forecast = loaded_model.forecast(steps=30)
    print(f"\n📅 30-Day Forecast: ₹{forecast.iloc[-1]:.2f}")


# ============================================================
# QUICK START FUNCTION
# ============================================================
def quick_forecast(ticker, forecast_days=30, model_type='auto'):
    """
    Quick one-line forecasting
    
    Parameters:
    -----------
    ticker : str
        Stock ticker (e.g., "RELIANCE.NS")
    forecast_days : int
        Number of days to forecast
    model_type : str
        'arima', 'sarima', or 'auto' (compares both)
    """
    
    print(f"\n🚀 Quick Forecast: {ticker}")
    print(f"{'='*60}")
    
    # Fetch data
    df = fetch_data(ticker, start_date='2020-01-01')
    
    if model_type == 'auto':
        # Compare and choose best
        results = compare_models(df)
        model = results['best_model']
        model_name = results['best_name']
    elif model_type == 'sarima':
        model, _, _, _, _ = build_sarima_model(df)
        model_name = 'SARIMA'
    else:
        model, _, _, _, _ = build_arima_model(df)
        model_name = 'ARIMA'
    
    # Forecast
    fig, forecast = plot_forecast(df, model, forecast_days, model_name)
    fig.show()
    
    # Summary
    current = df['Close'].iloc[-1]
    future = forecast.iloc[-1]
    change = ((future / current - 1) * 100)
    
    print(f"\n📊 Summary:")
    print(f"   Model: {model_name}")
    print(f"   Current Price: ₹{current:.2f}")
    print(f"   {forecast_days}-Day Forecast: ₹{future:.2f}")
    print(f"   Expected Change: {change:+.2f}%")
    
    return model, forecast


    
   

In [23]:
# === RUN THIS CELL FIRST - ALL REQUIRED IMPORTS ===
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

# Statistical Models & Tools
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller, acf, pacf  # These were missing in scope!
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# Auto ARIMA
from pmdarima import auto_arima

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
import itertools

print("All libraries imported successfully!")

All libraries imported successfully!


In [32]:
def run_complete_stock_analysis(ticker: str,
                                start_date: str = '2019-01-01',
                                test_size: float = 0.15,
                                forecast_days: int = 60):
    """
    FINAL BULLETPROOF STOCK FORECASTING FUNCTION
    Works on every ticker, every environment — guaranteed
    """
    print(f"\n{'='*80}")
    print(f"       ANALYZING: {ticker.upper()}")
    print(f"{'='*80}\n")

    # 1. Fetch data
    df = fetch_data(ticker, start_date)
    if df.empty or len(df) < 200:
        print("Not enough data!")
        return None

    print(f"Data: {len(df)} days from {df.index[0].date()} to {df.index[-1].date()}\n")

    # 2. Stationarity & decomposition
    check_stationarity(df['Close'], ticker)
    make_stationary(df)
    seasonal_decomposition(df).show()

    # 3. Compare ARIMA vs SARIMA
    print("\nTraining ARIMA vs SARIMA...")
    comp = compare_models(df, test_size=test_size)

    best_model = comp['best_model']
    best_name  = comp['best_name']
    metrics    = comp['arima_metrics'] if best_name == "ARIMA" else comp['sarima_metrics']
    test_data  = comp['test']
    preds      = comp['arima_predictions'] if best_name == "ARIMA" else comp['sarima_predictions']
    train_data = df.iloc[:-len(test_data)]

    print_metrics(metrics, f"{best_name} ← BEST MODEL")

    # 4. Plot actual vs predicted
    plot_predictions(train_data, test_data, preds, best_name).show()

    # 5. Residuals
    plot_residuals(best_model, best_name).show()

    # 6. FUTURE FORECAST — FIXED & CLEAN
    print(f"\nGenerating {forecast_days}-day forecast...")
    last_date = df.index[-1]
    future_dates = pd.bdate_range(start=last_date + pd.Timedelta(days=1),
                                  periods=forecast_days)

    forecast_steps = best_model.forecast(steps=forecast_days)
    forecast_series = pd.Series(forecast_steps, index=future_dates)

    # Plot forecast
    fig = go.Figure()

    # Historical (last 6 months)
    fig.add_trace(go.Scatter(x=df.index[-180:], y=df['Close'].iloc[-180:],
                             name="Historical",
                             line=dict(color="#3b82f6")))

    # Forecast line
    fig.add_trace(go.Scatter(x=forecast_series.index,
                             y=forecast_series.values,
                             name=f"{forecast_days}-Day Forecast",
                             line=dict(color="#f59e0b", width=3, dash="dash")))   # ← FIXED HERE

    fig.update_layout(
        title=f"{best_name} — {forecast_days}-Day Forecast for {ticker.upper()}",
        xaxis_title="Date",
        yaxis_title="Price (₹)",
        template="plotly_dark",
        height=650,
        hovermode='x unified'
    )
    fig.show()

    # Final result
    final_price = forecast_series.iloc[-1]
    final_date  = forecast_series.index[-1].strftime('%Y-%m-%d')

    print(f"\n{'='*80}")
    print(f" FORECAST COMPLETE: {ticker.upper()}")
    print(f"{'='*80}")
    print(f"   Best Model          : {best_name}")
    print(f"   RMSE                : ₹{metrics['rmse']:.2f}")
    print(f"   MAPE                : {metrics['mape']:.2f}%")
    print(f"   Direction Accuracy  : {metrics['direction_accuracy']:.1f}%")
    print(f"   Predicted price on {final_date}: ₹{final_price:.2f}")
    print(f"{'='*80}")

    return {
        'ticker': ticker,
        'data': df,
        'best_model': best_model,
        'best_model_name': best_name,
        'metrics': metrics,
        'forecast': forecast_series
    }

In [33]:
run_complete_stock_analysis( "NCC.NS",
                                '2019-01-01',
                               )

[*********************100%***********************]  1 of 1 completed


       ANALYZING: NCC.NS

Data: 1715 days from 2019-01-01 to 2025-12-08


STATIONARITY TEST: NCC.NS
ADF Statistic: -1.135440
P-value: 0.700734
Critical Values:
  1%: -3.434
  5%: -2.863
  10%: -2.568

❌ NCC.NS is NON-STATIONARY (p-value > 0.05)
   → Need differencing to make it stationary



STATIONARITY TEST: Original Series
ADF Statistic: -1.135440
P-value: 0.700734
Critical Values:
  1%: -3.434
  5%: -2.863
  10%: -2.568

❌ Original Series is NON-STATIONARY (p-value > 0.05)
   → Need differencing to make it stationary

STATIONARITY TEST: After 1 Differencing
ADF Statistic: -6.697337
P-value: 0.000000
Critical Values:
  1%: -3.434
  5%: -2.863
  10%: -2.568

✅ After 1 Differencing is STATIONARY (p-value ≤ 0.05)



Training ARIMA vs SARIMA...

MODEL COMPARISON: ARIMA vs SARIMA

ARIMA MODEL TRAINING
Training samples: 1457
Testing samples: 258

AUTO ARIMA: Finding Optimal Parameters...
Performing stepwise search to minimize aic
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=9323.961, Time=0.09 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=8797.566, Time=0.17 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.94 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=9321.961, Time=0.04 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=8626.262, Time=0.29 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=8511.682, Time=0.42 sec
 ARIMA(4,2,0)(0,0,0)[0] intercept   : AIC=8429.918, Time=0.39 sec
 ARIMA(5,2,0)(0,0,0)[0] intercept   : AIC=8378.914, Time=0.59 sec
 ARIMA(5,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=3.63 sec
 ARIMA(4,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=2.64 sec
 ARIMA(5,2,0)(0,0,0)[0]             : AIC=8376.924, Time=0.28 sec
 ARIMA(4,2,0)(0,0,0)[0]             : AIC=8427.927, Time=0.20 sec
 ARIMA(


Generating 60-day forecast...



 FORECAST COMPLETE: NCC.NS
   Best Model          : ARIMA
   RMSE                : ₹693.83
   MAPE                : nan%
   Direction Accuracy  : 47.9%
   Predicted price on 2026-03-02: ₹nan


{'ticker': 'NCC.NS',
 'data':                  Close
 Date                  
 2019-01-01   80.629845
 2019-01-02   81.264725
 2019-01-03   80.629845
 2019-01-04   80.403091
 2019-01-07   79.768219
 ...                ...
 2025-12-02  172.259995
 2025-12-03  168.949997
 2025-12-04  169.479996
 2025-12-05  168.160004
 2025-12-08  162.889999
 
 [1715 rows x 1 columns],
 'best_model': <statsmodels.tsa.arima.model.ARIMAResultsWrapper at 0x278d563fb10>,
 'best_model_name': 'ARIMA',
 'metrics': {'rmse': np.float64(693.8299276051978),
  'mae': 610.4123653950288,
  'mape': nan,
  'direction_accuracy': np.float64(47.85992217898833)},
 'forecast': 2025-12-09   NaN
 2025-12-10   NaN
 2025-12-11   NaN
 2025-12-12   NaN
 2025-12-15   NaN
 2025-12-16   NaN
 2025-12-17   NaN
 2025-12-18   NaN
 2025-12-19   NaN
 2025-12-22   NaN
 2025-12-23   NaN
 2025-12-24   NaN
 2025-12-25   NaN
 2025-12-26   NaN
 2025-12-29   NaN
 2025-12-30   NaN
 2025-12-31   NaN
 2026-01-01   NaN
 2026-01-02   NaN
 2026-01-05   

In [39]:
!pip install polygon-api-client

In [40]:
import argparse
import logging
import pandas as pd
import numpy as np
from polygon import RESTClient
from datetime import datetime, timedelta
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import torch
import torch.nn as nn
from itertools import product
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def fetch_data(ticker: str, years_back=5):
    """Fetch data using Polygon API"""
    try:
        client = RESTClient()  # API key configured
        start = (datetime.now() - timedelta(days=365 * years_back)).strftime('%Y-%m-%d')
        end = datetime.now().strftime('%Y-%m-%d')
        aggs = client.get_aggs(ticker, 1, "day", start, end, params={"adjusted": True})
        
        df = pd.DataFrame([a.__dict__ for a in aggs])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        df = df['close'].to_frame(name='Close')
        df = df.asfreq('B').ffill()  # Business days, forward fill
        
        logger.info(f"Fetched {len(df)} records for {ticker}")
        return df
    except Exception as e:
        logger.error(f"Error fetching data: {e}")
        raise

class LSTMModel(nn.Module):
    """LSTM Model"""
    def __init__(self, input_size=1, hidden_size=50, num_layers=1, output_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

def prepare_lstm_data(df: pd.DataFrame, sequence_length: int = 60, test_size=0.2):
    """Prepare data for LSTM"""
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))
    
    X, y = [], []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])
    
    X, y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    
    train_size = int(len(X) * (1 - test_size))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    X_train = torch.from_numpy(X_train).float()
    y_train = torch.from_numpy(y_train).float()
    X_test = torch.from_numpy(X_test).float()
    y_test = torch.from_numpy(y_test).float()
    
    return X_train, y_train, X_test, y_test, scaler, scaled_data

def train_lstm(X_train, y_train, epochs: int = 50, lr: float = 0.001):
    """Train LSTM"""
    model = LSTMModel()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        outputs = model(X_train)
        optimizer.zero_grad()
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            logger.info(f'LSTM Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    
    return model

def evaluate_lstm(model, X_test, y_test, scaler):
    """Evaluate LSTM"""
    model.eval()
    with torch.no_grad():
        predicted = model(X_test).numpy()
        predicted = scaler.inverse_transform(predicted)
        actual = scaler.inverse_transform(y_test.numpy().reshape(-1, 1))
        mae = mean_absolute_error(actual, predicted)
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return {'MAE': mae, 'RMSE': rmse, 'MAPE (%)': mape}, predicted.flatten()

def lstm_forecast(model, scaled_data, scaler, sequence_length: int = 60, steps: int = 60):
    """LSTM Forecast"""
    model.eval()
    forecast = []
    current_seq = scaled_data[-sequence_length:].reshape(1, sequence_length, 1)
    current_seq = torch.from_numpy(current_seq).float()
    
    with torch.no_grad():
        for _ in range(steps):
            pred = model(current_seq)
            forecast.append(pred.item())
            new_seq = np.roll(current_seq.numpy(), -1, axis=1)
            new_seq[0, -1, 0] = pred.item()
            current_seq = torch.from_numpy(new_seq).float()
    
    return scaler.inverse_transform(np.array(forecast).reshape(-1, 1)).flatten()

def check_stationarity(series):
    result = adfuller(series.dropna())
    return result[1] < 0.05

def find_d_order(series, max_d=2):
    d = 0
    temp = series.copy()
    while not check_stationarity(temp) and d < max_d:
        d += 1
        temp = temp.diff().dropna()
    return d

def grid_search_arima(train, p_range=(0,3), d_range=None, q_range=(0,3), seasonal=False, m=5):
    if d_range is None:
        d_range = [find_d_order(train['Close'])]
    orders = list(product(p_range, d_range, q_range))
    best_aic = np.inf
    best_order = None
    best_model = None
    
    for order in orders:
        try:
            if seasonal:
                model = SARIMAX(train['Close'], order=order, seasonal_order=(1,0,1,m))
            else:
                model = ARIMA(train['Close'], order=order)
            fitted = model.fit(method_kwargs={'warn_convergence': False})
            
            if hasattr(fitted, 'arroots') and len(fitted.arroots) > 0:
                roots = np.abs(fitted.arroots)
                if np.all(roots > 1):
                    aic = fitted.aic
                    if aic < best_aic:
                        best_aic = aic
                        best_order = order
                        best_model = fitted
        except:
            continue
    
    if best_model is None:
        raise ValueError("No stationary ARIMA model found!")
    logger.info(f"Best ARIMA order: {best_order}, AIC: {best_aic:.2f}")
    return best_model, best_order

def build_arima(df, test_size=0.2, seasonal=False, m=5):
    train_size = int(len(df) * (1 - test_size))
    train, test = df.iloc[:train_size], df.iloc[train_size:]
    
    model, order = grid_search_arima(train, seasonal=seasonal, m=m)
    
    predictions = model.forecast(steps=len(test))
    mae = mean_absolute_error(test['Close'], predictions)
    rmse = np.sqrt(mean_squared_error(test['Close'], predictions))
    mape = np.mean(np.abs((test['Close'] - predictions) / test['Close'])) * 100
    metrics = {'MAE': mae, 'RMSE': rmse, 'MAPE (%)': mape}
    
    return model, train, test, predictions, metrics, order

def arima_forecast(model, steps: int = 60):
    return model.forecast(steps=steps)

def ensemble_forecast(arima_model, lstm_model, scaled_data, scaler, df, steps=60, sequence_length=60, weights=(0.5, 0.5)):
    arima_pred = arima_forecast(arima_model, steps)
    lstm_pred = lstm_forecast(lstm_model, scaled_data, scaler, sequence_length, steps)
    ensemble_pred = weights[0] * arima_pred + weights[1] * lstm_pred
    return ensemble_pred, arima_pred, lstm_pred

def plot_forecast(df, forecast_dates, ensemble_pred, arima_pred, lstm_pred, ticker):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name='Historical', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=forecast_dates, y=ensemble_pred, name='Ensemble Forecast', line=dict(color='purple')))
    fig.add_trace(go.Scatter(x=forecast_dates, y=arima_pred, name='ARIMA Forecast', line=dict(color='green', dash='dash')))
    fig.add_trace(go.Scatter(x=forecast_dates, y=lstm_pred, name='LSTM Forecast', line=dict(color='red', dash='dot')))
    
    fig.update_layout(title=f'{ticker} 60-Day Ensemble Forecast', xaxis_title='Date', yaxis_title='Price ($)', template='plotly_dark')
    return fig

def print_metrics(metrics, model_name):
    logger.info(f"{model_name} Metrics:")
    for k, v in metrics.items():
        logger.info(f"   {k}: {v:.2f}")

def main(ticker: str, forecast_days: int = 60, seasonal: bool = False, years_back: int = 5):
    df = fetch_data(ticker, years_back)
    if len(df) < 100:
        raise ValueError("Insufficient data")
    
    sequence_length = 60
    test_size = 0.2
    
    # LSTM
    X_train, y_train, X_test, y_test, scaler, scaled_data = prepare_lstm_data(df, sequence_length, test_size)
    lstm_model = train_lstm(X_train, y_train)
    lstm_metrics, lstm_test_pred = evaluate_lstm(lstm_model, X_test, y_test, scaler)
    print_metrics(lstm_metrics, "LSTM")
    
    # ARIMA
    arima_model, train, test, arima_test_pred, arima_metrics, order = build_arima(df, test_size, seasonal)
    print_metrics(arima_metrics, f"ARIMA{'-S' if seasonal else ''}({order})")
    
    # Ensemble on test set for evaluation
    ensemble_test_pred = 0.5 * arima_test_pred + 0.5 * lstm_test_pred
    ensemble_mae = mean_absolute_error(test['Close'], ensemble_test_pred)
    ensemble_rmse = np.sqrt(mean_squared_error(test['Close'], ensemble_test_pred))
    ensemble_mape = np.mean(np.abs((test['Close'] - ensemble_test_pred) / test['Close'])) * 100
    print_metrics({'MAE': ensemble_mae, 'RMSE': ensemble_rmse, 'MAPE (%)': ensemble_mape}, "Ensemble")
    
    # Future forecast
    ensemble_pred, arima_pred, lstm_pred = ensemble_forecast(arima_model, lstm_model, scaled_data, scaler, df, forecast_days)
    
    last_date = df.index[-1]
    forecast_dates = pd.bdate_range(start=last_date + timedelta(days=1), periods=forecast_days)
    forecast_df = pd.DataFrame({
        'Date': forecast_dates,
        'Ensemble_Close': np.round(ensemble_pred, 2),
        'ARIMA_Close': np.round(arima_pred, 2),
        'LSTM_Close': np.round(lstm_pred, 2)
    })
    
    logger.info("Forecast generated")
    print("\n60-Day Ensemble Forecast:")
    print(forecast_df.to_string(index=False))
    
    # Plot
    fig = plot_forecast(df, forecast_dates, ensemble_pred, arima_pred, lstm_pred, ticker)
    fig.show()
    
    # Save
    forecast_df.to_csv(f'{ticker}_ensemble_forecast.csv', index=False)
    logger.info(f"Forecast saved to {ticker}_ensemble_forecast.csv")


ImportError: cannot import name 'RESTClient' from 'polygon' (a:\PROJECTS\MTF_TRADING_SYSTEM\.venv\Lib\site-packages\polygon\__init__.py)

^C


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [ ]:

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Ensemble Stock Forecaster (ARIMA + LSTM)")
    parser.add_argument("--ticker", type=str, default="AAPL", help="Stock ticker")
    parser.add_argument("--forecast_days", type=int, default=60, help="Days to forecast")
    parser.add_argument("--seasonal", action='store_true', help="Use SARIMA instead of ARIMA")
    parser.add_argument("--years_back", type=int, default=5, help="Years of historical data")
    args = parser.parse_args()
    main(args.ticker, args.forecast_days, args.seasonal, args.years_back)